In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.


In [2]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
print('y_train shape:', y_train.shape)


y_train shape: (50000, 10)


In [23]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10


    

def split_data_noniid(x,y,fractions):
    s = len(x)
    l = np.eye(10)
    sorted_ind=np.array([np.where([list(y_)==list(l[i]) for y_ in y])[0] for i in range(10)])
    #print("sorted ind shape: ", sorted_ind.shape)
    #for d in sorted_ind:
     #   print("d shape: ", d.shape)

        

    parts=2
    c = np.array([np.int32(np.linspace(0,len(sorted_ind[i]),parts+1)) for i in range(10)])
    #print("c shape: ", c.shape)
    #print("c: ", c)
    new_sorted_ind = np.array([[sorted_ind[i][c[i,j]:c[i,j+1]] for j in range(parts)] for i in range(10)])
    print("new sorted ind shape: ", new_sorted_ind.shape)
    #for inner in new_sorted_ind:
    #    for d in inner:
    #        print("d shape: ", d.shape)
    
    m = 10
#    x_list = [[[]]*10][0]
 #   y_list = [[[]]*10][0]
    
    x_list = np.empty(10,list)
    y_list = np.empty(10,list)
    
    for i in range(10):
        if i<9:
            j=i+1
        else:
            j=0   
        x_list[i] = np.concatenate((x[new_sorted_ind[i,0]],x[new_sorted_ind[j,1]]))
        y_list[i] = np.concatenate((y[new_sorted_ind[i,0]],y[new_sorted_ind[j,1]]))

    

    return np.array(x_list),np.array(y_list)



#F = [10,10,8,8,6,6,4,4,2,2,1,1]
#M = len(F)

x_train_splits, y_train_splits = split_data_noniid(x_train,y_train,fractions=10)
print("x_train_splits shape: ", x_train_splits.shape)
for x, y in zip(x_train_splits,y_train_splits):
    print("x shape: ", x.shape, ", y shape: ", y.shape)
#x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
#print("x_train shape: ", np.array(x_train).shape)
#tot_len = 0
#for split in x_train_splits:
#    print("x_train_splits shape: ", split.shape)
#    tot_len += len(split)
#print("tot len: ", tot_len)
alliance.set_validation_dataset(x_test,y_test)

classes = np.arange(10)
for i in range(M):
    member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
    alliance.add_member(member)

new sorted ind shape:  (10, 2, 2500)
x_train_splits shape:  (10,)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)
x shape:  (5000, 32, 32, 3) , y shape:  (5000, 10)


In [22]:
from cifarmodel import KerasSequentialCifar

base_learner = KerasSequentialCifar()
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


In [ ]:
parameters = {"nr_global_iterations": 300, "nr_local_iterations":1, "training_steps": None, "data_augmentation": True, "model_size_averaging": True} 
training_loss, test_loss = fedavg_model.fit(parameters)

fit starts!
alliance temp model:  None
alliance temp model:  <cifarmodel.KerasSequentialCifar object at 0x7fddf0d08080>
global epoch:  0
virtual memory used:  21.0 %
training starts model size:  5000
training steps:  None

Epoch 1/1
157/157 [==============================] - 12s 77ms/step - loss: 1.1842 - accuracy: 0.5136
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 10s 63ms/step - loss: 0.7750 - accuracy: 0.7668
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 55ms/step - loss: 1.3620 - accuracy: 0.4882
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 60ms/step - loss: 1.0873 - accuracy: 0.6258
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 10s 61ms/step - loss: 1.0334 - accuracy: 0.6260
training starts model size:  5000
training steps: 

157/157 [==============================] - 9s 60ms/step - loss: 0.6838 - accuracy: 0.7806
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 58ms/step - loss: 0.7021 - accuracy: 0.7776
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 55ms/step - loss: 0.6622 - accuracy: 0.8040
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 58ms/step - loss: 0.5270 - accuracy: 0.8394
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 53ms/step - loss: 0.6955 - accuracy: 0.7836
training time:  92.7054 s.
average model starts
average model ends:  0.4426 s.
alliance_test_loss
test_loss:  [0.1   0.129 0.121 0.155 0.157]
global epoch:  5
virtual memory used:  28.0 %
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [=======================

157/157 [==============================] - 8s 48ms/step - loss: 0.7485 - accuracy: 0.7200
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 55ms/step - loss: 0.5980 - accuracy: 0.7934
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 53ms/step - loss: 0.8047 - accuracy: 0.6788
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 59ms/step - loss: 0.5601 - accuracy: 0.8266
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 0.4867 - accuracy: 0.8454
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 52ms/step - loss: 0.7796 - accuracy: 0.7160
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 48ms/step - loss: 

Epoch 1/1
157/157 [==============================] - 7s 45ms/step - loss: 0.2841 - accuracy: 0.9122
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 42ms/step - loss: 0.7852 - accuracy: 0.6900
training time:  76.9785 s.
average model starts
average model ends:  0.3547 s.
alliance_test_loss
test_loss:  [0.1   0.129 0.121 0.155 0.157 0.168 0.208 0.241 0.279 0.326 0.302 0.314
 0.333 0.302]
global epoch:  14
virtual memory used:  30.5 %
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 49ms/step - loss: 0.7687 - accuracy: 0.6932
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 44ms/step - loss: 0.7247 - accuracy: 0.7340
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 44ms/step - loss: 0.6794 - accuracy: 0.7408
training starts model size:  500

157/157 [==============================] - 8s 49ms/step - loss: 0.6681 - accuracy: 0.7570
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 0.7224 - accuracy: 0.7192
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 52ms/step - loss: 0.3294 - accuracy: 0.9016
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 45ms/step - loss: 0.6295 - accuracy: 0.7616
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 46ms/step - loss: 0.2684 - accuracy: 0.9236
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 44ms/step - loss: 0.4178 - accuracy: 0.8746
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 52ms/step - loss: 

...

157/157 [==============================] - 9s 60ms/step - loss: 0.3248 - accuracy: 0.8796
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 0.1490 - accuracy: 0.9526
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 55ms/step - loss: 0.4852 - accuracy: 0.8072
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 54ms/step - loss: 0.4039 - accuracy: 0.8444
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 10s 61ms/step - loss: 0.2043 - accuracy: 0.9274
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 57ms/step - loss: 0.2350 - accuracy: 0.9224
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 59ms/step - loss:

Epoch 1/1
157/157 [==============================] - 7s 47ms/step - loss: 0.1608 - accuracy: 0.9524
training time:  81.1272 s.
average model starts
average model ends:  0.3299 s.
alliance_test_loss
test_loss:  [0.1   0.129 0.121 0.155 0.157 0.168 0.208 0.241 0.279 0.326 0.302 0.314
 0.333 0.302 0.325 0.353 0.342 0.349 0.369 0.381 0.355 0.366 0.349 0.393
 0.393 0.396 0.382 0.355 0.324 0.352 0.377 0.332 0.329 0.349 0.336 0.348
 0.334 0.338 0.37  0.309 0.322 0.353 0.306 0.34  0.302 0.322 0.32  0.327
 0.351 0.331 0.339 0.343 0.342 0.314 0.337 0.323 0.353 0.337 0.337 0.343
 0.331 0.343 0.336 0.328 0.338 0.359 0.347 0.335 0.365 0.334 0.349 0.37
 0.338 0.384 0.374 0.381 0.365 0.368 0.352 0.381 0.395 0.388 0.379 0.38
 0.4   0.403 0.394 0.396 0.399 0.387 0.407 0.381 0.392 0.387 0.41 ]
global epoch:  95
virtual memory used:  34.3 %
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 54ms/step - loss: 0.4871 - accuracy: 0.8110
training s

157/157 [==============================] - 7s 45ms/step - loss: 0.2036 - accuracy: 0.9354
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 46ms/step - loss: 0.1833 - accuracy: 0.9454
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 43ms/step - loss: 0.3914 - accuracy: 0.8502
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 53ms/step - loss: 0.3062 - accuracy: 0.8874
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 7s 46ms/step - loss: 0.2468 - accuracy: 0.9172
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 0.3719 - accuracy: 0.8650
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 54ms/step - loss: 

average model ends:  0.4169 s.
alliance_test_loss
test_loss:  [0.1   0.129 0.121 0.155 0.157 0.168 0.208 0.241 0.279 0.326 0.302 0.314
 0.333 0.302 0.325 0.353 0.342 0.349 0.369 0.381 0.355 0.366 0.349 0.393
 0.393 0.396 0.382 0.355 0.324 0.352 0.377 0.332 0.329 0.349 0.336 0.348
 0.334 0.338 0.37  0.309 0.322 0.353 0.306 0.34  0.302 0.322 0.32  0.327
 0.351 0.331 0.339 0.343 0.342 0.314 0.337 0.323 0.353 0.337 0.337 0.343
 0.331 0.343 0.336 0.328 0.338 0.359 0.347 0.335 0.365 0.334 0.349 0.37
 0.338 0.384 0.374 0.381 0.365 0.368 0.352 0.381 0.395 0.388 0.379 0.38
 0.4   0.403 0.394 0.396 0.399 0.387 0.407 0.381 0.392 0.387 0.41  0.379
 0.392 0.379 0.383 0.38  0.409 0.407]
global epoch:  102
virtual memory used:  33.8 %
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 53ms/step - loss: 0.3510 - accuracy: 0.8728
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56

157/157 [==============================] - 9s 55ms/step - loss: 0.4562 - accuracy: 0.8198
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 55ms/step - loss: 0.2455 - accuracy: 0.9120
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 53ms/step - loss: 0.2856 - accuracy: 0.8954
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 51ms/step - loss: 0.2020 - accuracy: 0.9342
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 8s 54ms/step - loss: 0.2155 - accuracy: 0.9278
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 0.3398 - accuracy: 0.8718
training starts model size:  5000
training steps:  None
Epoch 1/1
157/157 [==============================] - 9s 56ms/step - loss: 

In [ ]:
from matplotlib import pylab as plt
import pickle
pickle.dump(fedavg_model.test_loss,open('test_loss_noniid_1epochloc.p','wb'))


plt.plot(fedavg_model.test_loss)
plt.savefig('accuracy_progress_noniid_1epochloc')
plt.show()
